In [ ]:
import sys, os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 
sys.path.append('..')
import avstack
import avapi
from copy import copy, deepcopy


import numpy as np
%load_ext autoreload
%autoreload 2

%matplotlib inline

data_dir = '/data/spencer/CARLA/carla-object-v1/'
CSM = avapi.carla.CarlaScenesManager(data_dir, verbose=False)

In [ ]:
CSD = CSM.get_scene_dataset_by_name(CSM.scenes[-1])
print(len(CSD.sensor_file_post[list(CSD.sensor_file_post.keys())[0]]['frame']), 'camera frames')

frame_ordering_cam = list(CSD.sensor_file_post['CAM_FRONT']['frame'].keys())
frame_ordering_ego = list(CSD.ego_files['frame'].keys())
frame_ordering = [x for x in frame_ordering_cam if x in frame_ordering_ego]

# frame_ordering_inf = list(CSD.sensor_file_post['CAM_INFRASTRUCTURE_001']['frame'].keys())

In [ ]:
img.shape

In [ ]:
img.calibration.P

In [ ]:
# Get scene dataset
frame = frame_ordering[30]
all_sensors = ['CAM_FRONT']
for cam in all_sensors:
    if 'DEPTH' in cam:
        img = CSD.get_depthimage(frame, cam)
    else:
        img = CSD.get_image(frame, cam)
    calib = CSD.get_calibration(frame, cam)
    ego = CSD.get_ego(frame)
    ego.attitude = ego.attitude.T
    ego.box.rot = ego.box.rot.T
    pc = CSD.get_lidar(frame, 'LIDAR_TOP')
    
    # Get "local" objects and validate same|
    objects_local = CSD.get_objects(frame, cam)
    
    # visualize the results
    avapi.visualize.snapshot.show_image_with_boxes(img, objects_local, inline=True)
#     avapi.visualize.show_lidar_on_image(pc, img)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,10))

pc = CSD.get_lidar(frame, 'LIDAR_TOP')

sc = 50
plt.scatter(pc.data[:,1], pc.data[:,0], s=4)
plt.xlim([-sc, sc])
plt.ylim([-sc, sc])
# for det in dets:
#     plt.scatter(det.box3d.t[1], det.box3d.t[0], marker='x', s=30)
plt.gca().invert_xaxis()
plt.gca().set_aspect('equal', adjustable='box')

In [ ]:
%matplotlib inline
idx = 1
sens = f'CAM_INFRASTRUCTURE_00{idx}'
idx_frame = 300
frame_ordering_cam = list(CSD.sensor_file_post[sens]['frame'].keys())
frame_cam = frame_ordering_cam[idx_frame]
ego = CSD.get_ego(frame_cam)
img = CSD.get_image(frame_cam, sens)
objs = CSD.get_objects(frame_cam, sens)
avapi.visualize.show_image_with_boxes(img, objs, inline=True)

sens = f'LIDAR_INFRASTRUCTURE_00{idx}'
frame_ordering_lid = list(CSD.sensor_file_post[sens]['frame'].keys())
frame_lidar = min(frame_ordering_lid, key=lambda x:abs(x-frame_cam))
if abs(frame_cam-frame_lidar) < 2:
    pc = CSD.get_lidar(frame_lidar, sens)
#     pc.transform_tobjs.o_ground()  # even this works!
#     objs = CSD.get_objects(frame_cam, sens)
#     for obj in objs:
#         obj.change_origin(pc.origin)
    avapi.visualize.show_lidar_on_image(pc, img, boxes=objs)
else:
    print('Cannot find close frame...eventually add change origin to unwrap to global')

In [ ]:
# M = avstack.modules.perception.object2dfv.MMDetObjectDetector2D(
#     model='cascade_mask_rcnn', threshold=0.2, dataset='nuimages', gpu=0)

# M2D_I = avstack.modules.perception.object2dfv.MMDetObjectDetector2D(
#     model='fasterrcnn', threshold=0.5, dataset='carla-infrastructure', gpu=1)

# M3D = avstack.modules.perception.object3d.MMDetObjectDetector3D(
#     model='pointpillars', threshold=0.1, dataset='carla', epoch='latest', gpu=0)

M3D_I = avstack.modules.perception.object3d.MMDetObjectDetector3D(
    model='3dssd', threshold=0.1, dataset='carla-infrastructure', epoch='latest', gpu=1)

In [ ]:
pc_postproj = deepcopy(pc)
pc_postproj.transform_to_ground()
dets_3dI = M3D_I(pc_postproj, 'dets3d')
avapi.visualize.show_lidar_on_image(pc_postproj, img, boxes=dets_3dI, inline=True)

In [ ]:
dets_2dI = M2D_I(img, 'dets2d')
avapi.visualize.show_image_with_boxes(img, dets_2dI, inline=True)

In [ ]:
M = avstack.modules.perception.object2dfv.MMDetObjectDetector2D(
    model='cascade_mask_rcnn', threshold=0.5, dataset='nuimages', gpu=1)

In [ ]:
dets = M(img, 'dets2d')
avapi.visualize.show_image_with_boxes(img, dets, inline=True)

In [ ]:
M3D = avstack.modules.perception.object3d.MMDetObjectDetector3D(
    model='pointpillars', threshold=0.1, dataset='carla', epoch='latest', gpu=1)

In [ ]:
# frame = 800
pc = CSD.get_lidar(frame, 'LIDAR_TOP')
img_viz = CSD.get_image(frame, 'CAM_FRONT')
dets = M3D(pc, 'dets3d')
avapi.visualize.show_image_with_boxes(img_viz, dets, inline=True)

In [ ]:
pointpillars

In [ ]:
mmdet3D

In [ ]:
dets

In [ ]:
[d.score for d in dets]

### Point Cloud Visualization

In [ ]:
dets[0]

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,10))

sc = 50
plt.scatter(pc.data[:,1], pc.data[:,0], s=4)
plt.xlim([-sc, sc])
plt.ylim([-sc, sc])
for det in dets:
    plt.scatter(det.box3d.t[1], det.box3d.t[0], marker='x', s=30)
# plt.gca().invert_xaxis()
plt.gca().set_aspect('equal', adjustable='box')

## Test Perception

#### 2D Perception

In [ ]:
cam = 'CAM_FRONT'
imgs_pre = [CSD.get_image(frame, cam) for frame in CSD.get_frames(cam)[:50]]

M = avstack.modules.perception.object2dfv.MMDetObjectDetector2D(model='htc', threshold=0.7, dataset='nuimages')
%timeit -n 10 M(imgs_pre[np.random.randint(0, len(imgs_pre)-1)], '2d-dets') 

M = avstack.modules.perception.object2dfv.MMDetObjectDetector2D(model='cascade_mask_rcnn', threshold=0.7, dataset='nuimages')
%timeit -n 10 M(imgs_pre[np.random.randint(0, len(imgs_pre)-1)], '2d-dets') 

M = avstack.modules.perception.object2dfv.MMDetObjectDetector2D(model='fasterrcnn', threshold=0.7, dataset='nuscenes')
%timeit -n 10 M(imgs_pre[np.random.randint(0, len(imgs_pre)-1)], '2d-dets') 

#### 3D Perception

In [ ]:
M3D = avstack.modules.perception.object3d.MMDetObjectDetector3D(
    model='pointpillars', dataset='nuscenes', threshold=0.3, gpu=1)

In [ ]:
# NDM = NSM.get_scene_dataset_by_index(1)
frame = 12
pc = CSD.get_lidar(frame)
pc2 = deepcopy(pc)
pc2.data = np.concatenate((pc.data, np.ones((len(pc.data),2))), axis=1)
R_90 = avstack.transformations.rotz(np.pi/2)
pc2.calibration.origin = pc2.calibration.origin.rotate(R_90)
pc2.data[:,:3] = pc2.data[:,:3] @ R_90
res = M3D(pc2, 'lidar')

In [ ]:
img = CSD.get_image(frame, sensor='CAM_BACK')
avapi.visualize.show_image_with_boxes(img, res, inline=True)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,10))

sc = 20
plt.scatter(pc.data[:,1], pc.data[:,0], s=4)
plt.xlim([-sc, sc])
plt.ylim([-sc, sc])
plt.gca().invert_xaxis()
plt.gca().set_aspect('equal', adjustable='box')

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,10))

sc = 20
plt.scatter(pc2.data[:,1], pc2.data[:,0], s=4)
plt.xlim([-sc, sc])
plt.ylim([-sc, sc])
plt.gca().invert_xaxis()
plt.gca().set_aspect('equal', adjustable='box')